In [1]:
import numpy as np
import random
import torch
from src.dataloaders import Dataset100k
from src.models import MLPBCEModel
from src.trainer import Trainer
from src.metrics import hitratio, ndcg

np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(f"{device=}")

device='cuda'


In [2]:
class config:
    data_dir = 'ml-100k'
    epochs = 40
    batch_size = 2048
    dim = 32
    lr = 0.005
    weight_decay = 0.0001

dataset = Dataset100k(config.data_dir)
dataset.gen_adjacency()
dataset.make_train_test()
print(f"{dataset.train_size=}, {dataset.test_size=}")

metrics = {
    "HR@1": (hitratio, {"top_n": 1}),
    "HR@5": (hitratio, {"top_n": 5}),
    "HR@10": (hitratio, {"top_n": 10}),
    "NDCG@1": (ndcg, {"top_n": 1}),
    "NDCG@5": (ndcg, {"top_n": 5}),
    "NDCG@10": (ndcg, {"top_n": 10}),
}

dataset.train_size=198114, dataset.test_size=943


In [3]:
model = MLPBCEModel(dataset.user_count, dataset.item_count, embed_size=config.dim, layers=[32, 16, 8])

optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)

trainer = Trainer(dataset, model, optimizer, metrics, 
                  epochs=config.epochs, batch_size=config.batch_size,
                  device=device)

In [12]:
trainer.train(evaluate=True, verbose=True, progressbar=True)
# trainer.test(verbose=False, pbar=False)

Epoch 0: Avg Loss/Batch 0.292200            


HR@1: 0.13149522799575822
HR@5: 0.3828207847295864
HR@10: 0.5471898197242842
NDCG@1: 0.13149522799575822
NDCG@5: 0.25693256483407984
NDCG@10: 0.3106867825003414


Epoch 1: Avg Loss/Batch 0.291306            


HR@1: 0.1325556733828208
HR@5: 0.3828207847295864
HR@10: 0.5471898197242842
NDCG@1: 0.1325556733828208
NDCG@5: 0.25732394367440703
NDCG@10: 0.3111636191674662


Epoch 2: Avg Loss/Batch 0.290412            


...

HR@1: 0.12513255567338283
HR@5: 0.37327677624602335
HR@10: 0.545068928950159
NDCG@1: 0.12513255567338283
NDCG@5: 0.2524181059191822
NDCG@10: 0.3083012243762159


Epoch 38: Avg Loss/Batch 0.263212            


HR@1: 0.1261930010604454
HR@5: 0.3753976670201485
HR@10: 0.5482502651113468
NDCG@1: 0.1261930010604454
NDCG@5: 0.25313933779544046
NDCG@10: 0.30887655197912994


Epoch 39: Avg Loss/Batch 0.262561            


HR@1: 0.12513255567338283
HR@5: 0.37327677624602335
HR@10: 0.5471898197242842
NDCG@1: 0.12513255567338283
NDCG@5: 0.251788642552101
NDCG@10: 0.3080984164361115


In [ ]:
best_epoch = np.argmax([r["NDCG@10"] for r in trainer.test_log])
print(f"{best_epoch}: {trainer.test_log[best_epoch]}")

104: {'HR@1': 0.13679745493107104, 'HR@5': 0.3860021208907741, 'HR@10': 0.5535524920466596, 'NDCG@1': 0.13679745493107104, 'NDCG@5': 0.26183027654913826, 'NDCG@10': 0.3160622946925831}


In [ ]:
best_epoch = np.argmax([r["NDCG@10"] for r in trainer.test_log])
print(f"{best_epoch}: {trainer.test_log[best_epoch]}")

77: {'HR@1': 0.1357370095440085, 'HR@5': 0.37751855779427357, 'HR@10': 0.5556733828207847, 'NDCG@1': 0.1357370095440085, 'NDCG@5': 0.25860058009897624, 'NDCG@10': 0.3158008204897394}


In [ ]:
best_epoch = np.argmax([r["NDCG@10"] for r in trainer.test_log])
print(f"{best_epoch}: {trainer.test_log[best_epoch]}")

25: {'HR@1': 0.1357370095440085, 'HR@5': 0.34994697773064687, 'HR@10': 0.5471898197242842, 'NDCG@1': 0.1357370095440085, 'NDCG@5': 0.24658333238245064, 'NDCG@10': 0.30979900867095866}


In [ ]:
torch.save(trainer.model.state_dict(), "saved_models/nmfbce.pt")
# trainer.model.load_state_dict(torch.load("saved_models/nmfbce.pt"))